In [ ]:
metric_name = 'maniqa'

In [ ]:
metric_path = f'./subjects/{metric_name}'

In [ ]:
!git submodule update --init --recursive --depth=1 {metric_path}

In [ ]:
!cd {metric_path} && docker build --tag {metric_name} .

In [ ]:
attack_list = [
    'korhonen-et-al',
    'madc',
    'amifgsm',
    'mifgsm',
    'ifgsm',
    'std-fgsm',
    'cumulative-uap',
    'uap',
    'generative-uap'
]
trainable_attacks = [
    'cumulative-uap',
    'uap',
    'generative-uap'
]

In [ ]:
should_train = False

In [ ]:
import shutil
import os
import json

In [ ]:
with open(f'{metric_path}/config.json') as json_file:
        config = json.load(json_file)
        metric_model = config['weight']
        module = config['module']
        is_fr = config['is_fr']
quality_param = '--jpeg-quality 80' if is_fr else ''

In [ ]:
DATASETS_STORAGE = os.path.abspath('./data')
RESULTS_STORAGE = os.path.abspath('./res')
utils_path = os.path.abspath(fr'./methods/utils/')

In [ ]:
train_datasets = ' '.join(['VOC2012', 'COCO'])
train_datasets_paths = ' '.join(['/train/VOC2012', '/train/COCO_train_9999'])
test_datasets = ' '.join(['NIPS', 'DERF', 'VIMEO'])
test_datasets_paths = ' '.join(['/test/NIPS2017', '/test/DERF_blue_sky', '/test/vimeo_test_2001'])

In [ ]:
for attack_name in attack_list:
    print(attack_name) 
    attack_path = os.path.abspath(fr'./methods/{attack_name}/')
    !cd {attack_path} && docker build -t {attack_name}:{metric_name} --build-arg METRIC_IMAGE={metric_name} .
    if attack_name in trainable_attacks:
        if should_train:
            print('train')
            !docker run --gpus device=0 -i --rm \
              -v {DATASETS_STORAGE}:"/train":ro \
              -v {RESULTS_STORAGE}:"/artifacts" \
              -v {attack_path}/train.py:"/train.py" \
              -v {utils_path}/bounds.json:"/bounds.json" \
              -v {utils_path}/evaluate.py:"/evaluate.py" \
              -v {utils_path}/metrics.py:"/metrics.py" \
              -v {utils_path}/read_dataset.py:"/read_dataset.py" \
              {attack_name}:{metric_name} \
              python /train.py \
                --batch-size 1 \
                --path-train {train_datasets_paths} \
                --metric  {metric_name} \
                --train-dataset {train_datasets} \
                --save-dir /artifacts \
                {quality_param} \
                --device "cuda:0"
            for train_dataset in train_datasets.split():
                os.rename(
                    os.path.join(RESULTS_STORAGE, f'{train_dataset}.npy'),
                    os.path.join(RESULTS_STORAGE, f'{attack_name}_{metric_name}_{train_dataset}.npy')
                )
                os.rename(
                    os.path.join(RESULTS_STORAGE, f'{train_dataset}.png'),
                    os.path.join(RESULTS_STORAGE, f'{attack_name}_{metric_name}_{train_dataset}.png')
                )
        print('test')
        
        uap_paths = ' '.join([
            f'/artifacts/{attack_name}_{metric_name}_{train_dataset}.npy'
            for train_dataset in train_datasets.split()
        ])
        !docker run --gpus device=0 -i --rm \
          -v {DATASETS_STORAGE}:"/test":ro \
          -v {RESULTS_STORAGE}:"/artifacts" \
          -v {attack_path}/run.py:"/run.py" \
          -v {utils_path}/bounds.json:"/bounds.json" \
          -v {utils_path}/evaluate.py:"/evaluate.py" \
          -v {utils_path}/metrics.py:"/metrics.py" \
          -v {utils_path}/read_dataset.py:"/read_dataset.py" \
          {attack_name}:{metric_name} \
          python /run.py \
            --amplitude 0.2 0.4 0.8 \
            --metric  {metric_name} \
            --uap-path {uap_paths} \
            --train-dataset {train_datasets} \
            --test-dataset {test_datasets} \
            --dataset-path {test_datasets_paths} \
            {quality_param} \
            --save-path /artifacts/{attack_name}_{metric_name}_test.csv \
            --device "cuda:0"
    else:
        print('test')
        !docker run --gpus device=0 -i --rm \
          -v {DATASETS_STORAGE}:"/test":ro \
          -v {RESULTS_STORAGE}:"/artifacts" \
          -v {attack_path}/run.py:"/run.py" \
          -v {utils_path}/bounds.json:"/bounds.json" \
          -v {utils_path}/evaluate.py:"/evaluate.py" \
          -v {utils_path}/metrics.py:"/metrics.py" \
          -v {utils_path}/read_dataset.py:"/read_dataset.py" \
          {attack_name}:{metric_name} \
          python /run.py \
            --test-dataset {test_datasets} \
            --metric  {metric_name} \
            --dataset-path {test_datasets_paths} \
            {quality_param} \
            --save-path /artifacts/{attack_name}_{metric_name}_test.csv \
            --device "cuda:0"

